In [1]:
#Initialization
import numpy as np #Package for array functions
import formFactorFit
from numpy import random as rand
from numpy import sin, cos

The decay length for the heavy neutral lepton is 

$\lambda = R_{\oplus} \big[\, \dfrac{1.97 \times 10^{-9} MeV^{-1}}{d} \big]\,^2 \big[\, \dfrac{1 MeV}{m_N} \big]\,^4 \big[\, \dfrac{E_N}{10 MeV} \big]\, \sqrt{\dfrac{1-m_N^2/E_N^2}{0.99}}$ 

Where $R_{\oplus}$ is the radius of Earth, $d$ is the dipole coupling, $m_N$ is the mass of the lepton, and $E_N$ is the energy of the lepton.

In [2]:
def decay_length(d,mn,En):
    '''
    Find the characteristic decay length of the lepton
    
    args:
        d: dipole coupling constant in MeV^-1 (float)
        mn: mass of the lepton in GeV (float)
        En: Energy of the lepton in GeV (float or array of floats)
    
    returns:
        Lambda: Characteristic decay length in cm (float or array of floats, equal to the number of energies)
        
    action:
        Calculates characteristic decay length according to Plestid, 2021
    '''
    R_Earth = 6378.1 * 1000* 100    #Radius of the Earth (cm)
    mn_MeV = mn*1000  #Convert mass to MeV
    En_MeV = En*1000  #Convert energy to MeV
    Lambda = (R_Earth * (1.97e-9/d)**2 *(1/mn_MeV)**4 * (En_MeV/10) 
              * np.sqrt((1-mn**2/En**2)/0.99) )#Decay lengths (cm)
    
    return(Lambda)

We now wish to calculate $\dfrac{d\sigma}{d\cos{\Theta}}$, where $\Theta$ is the scattering angle, which will be used in the probability to scatter. We have a function for coherent scattering $\dfrac{d\sigma_{coh}}{dt}$ where $t = -q^2$ and $q$ is the transferred momentum. We will work in the infinite mass limit, so the energy transfered to the nucleus is zero, and $E_N = E_{\nu}$. Therefore, we have

$t = -(\overrightarrow{p_{\nu}} - \overrightarrow{p_{N}})^2 = E_{\nu}^2+p_N^2-2E_{\nu}p_N \cos\Theta$

Where we know, working in the lab frame

$p_N = \sqrt{E_N^2-m_N^2} = \sqrt{E_{\nu}^2-m_N^2}$  

$t = 2E_{\nu}^2 - m_N^2 - 2 E_{\nu}\sqrt{E_{\nu}^2-m_N^2} \cos\Theta $  

$dt = -2 E_{\nu} \sqrt{E_{\nu}^2 - m_N^2} d(\cos\Theta)$

Now, subbing in our differential cross section:

$\dfrac{d\sigma_{coh}}{dt} = \dfrac{Z^2 d^2 \alpha}{t E_{\nu}^2} (4E^2_{\nu} -m^2_{N}+\dfrac{m^4_N}{t})$

$\dfrac{d\sigma_{coh}}{d(-2E_{\nu}\sqrt{E_{\nu}^2 - m_N^2}\cos\Theta)} = \dfrac{Z^2 d^2 \alpha}{t E_{\nu}^2} (4E^2_{\nu} -m^2_{N}+\dfrac{m^4_N}{t})$

$\dfrac{d\sigma_{coh}}{d(\cos\Theta)} = \dfrac{-2\sqrt{E_{\nu}^2-m_N^2}*Z^2 d^2 \alpha}{E_{\nu}t} (4E^2_{\nu} -m^2_{N}+\dfrac{m^4_N}{t})$

$\dfrac{d\sigma_{coh}}{d(\cos\Theta)} = \dfrac{-2\sqrt{E_{\nu}^2-m_N^2}*Z^2 d^2 \alpha}{E_{\nu} (2E_{\nu}^2 - m_N^2 - 2 E_{\nu}\sqrt{E_{\nu}^2-m_N^2} \cos\Theta)} (4E^2_{\nu} -m^2_{N}+\dfrac{m^4_N}{2E_{\nu}^2 - m_N^2 - 2 E_{\nu}\sqrt{E_{\nu}^2-m_N^2} \cos\Theta})$

This returns a negative answer, so we flip the sign

In [3]:
def d_sigma_d_cos_Theta_coherent(d,mn,En,cos_Theta,Zed):
    '''
    Determine the differential cross section for a coherent scattering at a specified angle
    
    args:
        d: dipole coupling constant in MeV^-1 (float)
        mn: mass of the lepton in GeV (float)
        En: Energy of the lepton in GeV (float or array of floats)
        cos_Theta: cosine of the scattering angle (float, same size as En)
        Zed: Atomic number (int)
        
    returns:
        d_sigma_d_cos_Theta: differential cross section in cm of the scattering
                            (float, same size as En)
    
    actions:
        Computes the differential cross section in terms of MeV^{-2}, then
        converts it to cm^2
    '''
    alpha = 1/137 #Fine structure constant (SI Units)
    En_MeV = En *1000  #Neutrino/Lepton Energies in MeV
    mn_MeV = mn*1000  #Lepton mass in MeV
    t = (2*En_MeV**2 - mn_MeV**2 - 2*En_MeV*np.sqrt(En_MeV**2 - mn_MeV**2) 
          * cos_Theta) #Transfered momentum^2 (MeV^2)

    leading_terms = (-2* np.sqrt(En_MeV**2 - mn_MeV**2) * Zed**2 * d**2 * alpha)/(En_MeV*t) #MeV^-4
    second_terms = (4*En_MeV**2 - mn_MeV**2 + mn_MeV**4/t) #MeV^2
    Inv_Mev_to_cm = (197.3) * 1e-13 # (MeV^-1 to fm) * (fm to cm)
    d_sigma_d_cos_Theta = Inv_Mev_to_cm**2 * -leading_terms*second_terms #cm^2
    
    return(d_sigma_d_cos_Theta)

Not all interactions are coherent, and the Earth is not made of a single element.  To account for this, we calculate

$\dfrac{d\sigma}{d\cos\Theta} = \sum_Z f_Z \dfrac{d\sigma_{coh}(Z=1)}{d\cos\Theta} Z^2 F^2(q)$

Where $f_Z$ is the fractional number density of the nucleus ($n_Z/n_{tot}$), and $F^2(q)$ is the form factor.

In another module (formFactorFit.py), we fit Helm Form Factor parameters $R_1$ and $s$ and define a function to calculate the Helm Form Factor given these parameters and a momentum transfer.

In [4]:
def Full_d_sigma_d_cos_Theta(d, mn, En, cos_Theta, Zeds, R1s, Ss, fracs):
    '''
    Determine the differential cross section for a coherent and incoherent 
    scattering at a specified angle
    
    args:
        d: dipole coupling constant in MeV^-1 (float)
        mn: mass of the lepton in GeV (float)
        En: Energy of the lepton in GeV (float or array of floats)
        cos_Theta: cosine of the scattering angle (float, same size as En)
        Zeds: Atomic numbers (array of ints)
        R1s: Helm effective radii in fm (array of floats, same size as Zeds)
        Ss: Helm skin thicknesses in fm (array of floats, same size as Zeds)
        fracs: fractional number density of the nucleus in question, should sum to 1
                (array of floats, same size as Zeds)
        
    returns:
        d_sigma_d_cos_Theta: differential cross section in cm of the scattering
                            (float, same size as En)
    '''
    #Calculate the transfered momentum
    En_MeV = En *1000  #Neutrino/Lepton Energies in MeV
    mn_MeV = mn*1000  #Lepton mass in MeV
    t = (2*En_MeV**2 - mn_MeV**2 - 2*En_MeV*np.sqrt(En_MeV**2 - mn_MeV**2) 
          * cos_Theta) #Transfered momentum^2 (MeV^2)
    
    q = np.sqrt(t) #Transfered momentum (MeV)
    
    #Calculate the coherent cross sections for Z = 1
    d_sig_d_cos_coh = d_sigma_d_cos_Theta_coherent(d,mn,En,cos_Theta,1)
    
    #Initialize the cross section as 0
    d_sigma_d_cos_Theta = 0
    
    #Iterate through the nuclei
    for Zed_index in range(len(Zeds)):
        Zed = Zeds[Zed_index] #Atomic number
        R1 = R1s[Zed_index] #Effective nuclear radius
        s = Ss[Zed_index]  #Skin thickness
        frac = fracs[index]  #fractional number density of the nucleus
        
        FF2 = formFactorFit.HelmFF2(q,R1,s) #Form factors^2 for the transferred momentum
        
        d_sigma_d_cos_Theta += frac * d_sig_d_cos_coh * Zed**2 * FF2
    
    return(d_sigma_d_cos_Theta)

## Sampling Scattering Angles

We would like to preferentially sample our scattering angles such that we get events with the highest cross sections.  In our case, $\dfrac{d\sigma}{d\cos\Theta} \sim \dfrac{1}{1-\cos\Theta}$ roughly.  To sample according to this inverse relationship, we will let $\chi \sim U[0,1]$ and

$\cos\Theta = 1 - (1-\cos(\Theta)_{max})^{1-\chi} (1-\cos(\Theta)_{min})^{\chi}$

where $\cos(\Theta)_{min} = -1$ and $\cos(\Theta)_{max} = 1-\epsilon$ and $\epsilon << 1$.  This is done to prevent zeros in the denominator

In [5]:
def Sample_cos_Theta(Num_Events, epsilon):
    '''
    Samples cosines of the scattering angles according to a 1/(1-cos(Theta)) distribution
    
    args:
        Num_Events: number of scattering angles that we wish to sample (int)
        epsilon: how close the max value of cos(Theta) can be to 1 (float)
    
    returns:
        cos_Thetas: sampled scattering angles (array of length Num_Events, floats)
    '''
    cos_theta_min = -1 #Minimum possible cos(Theta) value
    cos_theta_max = 1 - epsilon #Maximum possible cos(Theta) value
    
    chi = rand.rand(Num_Events) #uniform random number between 0 and 1

    cos_Thetas = 1 - (1-cos_theta_max)**(1-chi) * (1-cos_theta_min)**chi
    return(cos_Thetas)

To account for our sampling bias, we must add a weight to the events when calculating the differential for the integral.  Our differential will be

$d\cos\Theta = \dfrac{2}{\sqrt[3]{n_{events}}} \dfrac{1-\cos\Theta}{(1-\cos\Theta)_{char}}$

where

$(1-\cos\Theta)_{char} = \dfrac{\cos(\Theta)_{max} - \cos(\Theta)_{min}}{\ln\big(\,(1-\cos(\Theta)_{min})/(1-\cos(\Theta)_{max} \big)\, } = \dfrac{2-\epsilon}{\ln(2/\epsilon)}$

In [6]:
def cos_Theta_differential(cos_Theta, epsilon, num_Events):
    '''
    Find the weighted differential for the scattering angle when performing the integral
    args:
        cos_Theta: cosine of the scattering angle (float or array of floats)
        epsilon: 1 - maximum possible value of cos(Theta) (float)
        num_Events: number of events for which we are performing the integral (int)
        
    returns:
        d_cos_Theta: Weighted scattering angle differential for the events 
                    (float or array of floats, same size as cos_Theta)
    '''
    char_val = (2-epsilon)/ np.log(2/epsilon)  #Characteristic value of 1 - cos(Theta)
    d_cos_Theta = (2/num_Events**(1/3)) * (1-cos_Theta)/char_val #Differential for the scattering angle
    
    return(d_cos_Theta)

## Arbitrary Cross-Section

If we are given an arbitrary differential cross-section distribution $\dfrac{1}{\sigma} \dfrac{d\sigma}{d\cos\Theta}$ how should we sample angles?

To be most effective, we want $\rho(\cos\Theta) = \dfrac{1}{\sigma} \dfrac{d\sigma}{d\cos\Theta}$ (this is already normalized).  If we have a function for the differential cross-section, we can just take the integral as the cdf and work from there.

If instead we just have the value at a discrete number of scattering angles, we can define
$cdf_i = \sum\rho(\cos\Theta_i) \Delta(\cos\Theta_i)$

After finding the cdf at the given points, sample $X \sim U[0,1]$ and find the neighboring points such that $cdf_i \leq X < cdf_{i+1}$.  We can then perform a linear extrapolation from these two points to find the corresponding value of $\cos\Theta$

In [7]:
def Sample_Arbitrary_Scat_Angles(num_events,cos_vals,frac_diff_cross_sec_vals):
    '''
    This function samples scattering angles to best fit a
    distribution of scattering cross-sections
    
    args:
        num_events: number of samples we want (int)
        cos_vals: cosines of the scattering angles at which we have
                data about the cross section (array of floats)
        frac_diff_cross_sec_vals: values of 1/sigma * d_simga/d_cos(Theta)
                at the specified values of cos(Theta) (array of floats, same size as cos_vals)
        
    returns:
        cos_Thetas: array of length num_Events with the sampled scattering
                cross sections.
                
    actions:
        calculates the cdf of the cross section at each angle, selects a
        random value between 0 and 1, finds the angle which has the 
        corresponding cdf
    '''
    #Create a new array for the cosines with -1 and 1 added
    cos_full = np.zeros(len(cos_vals)+2)
    cos_full[0],cos_full[-1] = -1,1
    cos_full[1:-1] = cos_vals
    
    #Create a new array for differential cross sections, extending the current edges to -1 and 1
    cross_sec_full = np.zeros(len(frac_diff_cross_sec_vals)+2)
    cross_sec_full[0], cross_sec_full[-1] = frac_diff_cross_sec_vals[0], frac_diff_cross_sec_vals[-1]
    cross_sec_full[1:-1] = frac_diff_cross_sec_vals
    
    #Create an array for the cdfs
    cdfs = np.zeros(len(cos_full))
    for i in range(1,len(cdfs)):
        cdfs[i] = np.trapz(cross_sec_full[:i+1],cos_full[:i+1])
    '''
    fig1 = plt.figure()
    plt.plot(cos_full, cdfs)
    '''
    #Uniformly sample the cdf
    Ran = rand.rand(num_events)
    #Interpolate to find the corresponding cosine value
    cos_Thetas = np.interp(Ran,cdfs,cos_full)
    return(cos_Thetas)

We will want to weight this arbitrary sampling, which can be done by

$\dfrac{1}{\sqrt[3]{n_{events}}} \dfrac{1}{\rho(\cos\Theta)} $

We will linearly interpolate the values of $\dfrac{1}{\sigma} \dfrac{d\sigma}{d\cos\Theta}$ to find the weight at each value of $\cos(\Theta)$.

In [8]:
def Arbitrary_Scattering_Differential(cos_Theta,num_events,cos_vals,frac_diff_cross_sec_vals):
    '''
    This function determines the proper differential for each event
    to perform the integral.
    args:
        cos_Theta: scattering angle of the event(s) (float or array of floats)
        num_events: number of samples we want (int)
        cos_vals: cosines of the scattering angles at which we have
                data about the cross section (array of floats)
        frac_diff_cross_sec_vals: values of 1/sigma * d_simga/d_cos(Theta)
                at the specified values of cos(Theta) (array of floats, same size as cos_vals)
                
    returns:
        d_cos_Theta: properly weighted differential (float of same size as cos_Theta)
    '''
    rho = np.interp(cos_Theta,cos_vals,frac_diff_cross_sec_vals) #Weight at scattering angle
    d_cos_Theta = 1/ (rho * num_events**(1/3))
    
    return(d_cos_Theta)